<a href="http://cocl.us/pytorch_link_top">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/Pytochtop.png" width="750" alt="IBM Product ">
</a> 


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork/image/IDSN-logo.png" width="200" alt="cognitiveclass.ai logo">


<h1>Objective</h1><ul><li> How to use linear classifier in pytorch.</li></ul> 


<h1>Linear  Classifier with PyTorch </h1>


<p>Before you use a  Deep neural network to solve the classification problem,  it 's a good idea to try and solve the problem with the simplest method. You will need the dataset object from the previous section.
In this lab, we solve the problem with a linear classifier.
 You will be asked to determine the maximum accuracy your linear classifier can achieve on the validation data for 5 epochs. We will give some free parameter values if you follow the instructions you will be able to answer the quiz. Just like the other labs there are several steps, but in this lab you will only be quizzed on the final result. </p>


<h2>Table of Contents</h2>


<div class="alert alert-block alert-info" style="margin-top: 20px">


<ul>
    <li><a href="#auxiliary"> Imports and Auxiliary Functions </a></li>
    <li><a href="#download_data"> Download data</a></li>
    <li><a href="#data_class"> Dataset Class</a></li>
    <li><a href="#trasform_Data_object">Transform Object and Dataset Object</a></li>
    <li><a href="#Question">Question</a></li>
</ul>
<p>Estimated Time Needed: <strong>25 min</strong></p>
 </div>
<hr>


<h2 id="auxiliary">Imports and Auxiliary Functions</h2>


The following are the libraries we are going to use for this lab:


In [1]:
%pip install torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 49.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 67.7 MB/s eta 0:00:00a 0:00:01
  Using cached torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 54.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 13.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 8.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 26.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim 
#import skillsnetwork 

<h2 id="download_data">Download Data</h2>


In this section, you are going to download the data from IBM object storage using **skillsnetwork.prepare** command. <b>skillsnetwork.prepare</b> is a command that's used to download a zip file, unzip it and store it in a specified directory. Locally we store the data in the directory  **/resources/data**. 


First, we download the file that contains the images:


In [1]:
wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
unzip concrete_crack_images_for_classification.zip

<h2 id="data_class">Dataset Class</h2>


In this section, we will use the previous code to build a dataset class. As before, make sure the even samples are positive, and the odd samples are negative.  In this case, if the parameter <code>train</code> is set to <code>True</code>, use the first 10 000 samples as training data; otherwise, the last 10 000 samples will be used as validation data. Do not forget to sort your files so they are in the same order.  


**Note:** We are using the first 10,000 samples as our training data instead of the available 30,000 to decrease the training time of the model. If you want, you can train it yourself with all 30,000 samples just by modifying 2 lines in the following code chunk.


In [4]:
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="data"
        positive="Positive"
        negative="Negative"

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=self.all_files[0:10000] #Change to 30000 to use the full test dataset
            self.Y=self.Y[0:10000] #Change to 30000 to use the full test dataset
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)    
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        
        
        image=Image.open(self.all_files[idx])
        y=self.Y[idx]
          
        
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

<h2 id="trasform_Data_object">Transform Object and Dataset Object</h2>


Create a transform object, that uses the <code>Compose</code> function. First use the transform <code>ToTensor()</code> and followed by <code>Normalize(mean, std)</code>. The value for <code> mean</code> and <code>std</code> are provided for you.


In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# transforms.ToTensor()
#transforms.Normalize(mean, std)
#transforms.Compose([])

transform =transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean, std)])


In [6]:
!ls 

3.1_linearclassiferPytorch.ipynb	      images
LICENSE.md				      learning-PyTorch-on-IPU
README_first.ipynb			      schnet-graph-network
concrete_crack_images_for_classification.zip  setup.sh
data					      temporal-graph-networks
distributed-kge				      useful-tips
finetuning-bert				      vit-model-training


Create object for the training data  <code>dataset_train</code> and validation <code>dataset_val</code>. Use the transform object to convert the images to tensors using the transform object:


In [7]:
dataset_train=Dataset(transform=transform,train=True)
dataset_val=Dataset(transform=transform,train=False)

We  can find the shape of the image:


In [8]:
dataset_train[0][0].shape

torch.Size([3, 227, 227])

We see that it's a color image with three channels:


In [9]:
size_of_image=3*227*227
size_of_image

154587

<h2 id="Question"> Question <h2>


<b> Create a custom module for Softmax for two classes,called model. The input size should be the <code>size_of_image</code>, you should record the maximum accuracy achieved on the validation data for the different epochs. For example if the 5 epochs the accuracy was 0.5, 0.2, 0.64,0.77, 0.66 you would select 0.77.</b>


Train the model with the following free parameter values:


<b>Parameter Values</b>
   <li>learning rate:0.1 </li>
   <li>momentum term:0.1 </li>
   <li>batch size training:5</li>
   <li>Loss function:Cross Entropy Loss </li>
   <li>epochs:5</li>
   <li>set: torch.manual_seed(0)</li>


In [10]:
torch.manual_seed(0)

<b>Custom Module:</b>


In [27]:
# Create logistic_regression class

class logistic_regression(nn.Module):
    
    # Constructor
    def __init__(self, n_inputs):
        super(logistic_regression, self).__init__()
        self.linear = nn.Linear(n_inputs, 2)
        
    # Prediction
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        yhat = torch.sigmoid(self.linear(x))
        return yhat

<b>Model Object:</b>


In [28]:
model = logistic_regression(size_of_image)

<b>Optimizer:</b>


In [29]:
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum=0.1)

<b>Criterion:</b>


In [30]:
criterion = nn.CrossEntropyLoss()

<b>Data Loader Training and Validation:</b>


In [31]:
train_loader = DataLoader(dataset = dataset_train, batch_size = 5)
validation_loader = DataLoader(dataset = dataset_val, batch_size = 5)

<b>Train Model with 5 epochs, should take 35 minutes: </b>


In [32]:
import time

n_epochs=100
loss_list=[]
accuracy_list=[]
correct=0
N_test=len(dataset_val)
N_train=len(dataset_train)
start_time = time.time()
#n_epochs

Loss=0
start_time = time.time()
for epoch in range(n_epochs):
    b_no=0
    loss_t = 0
    for x, y in train_loader:

        model.train() 
        b_no+=1
        #clear gradient 
        optimizer.zero_grad()
        
        #make a prediction 
        yhat=model(x)
        #print(yhat, yhat.shape)

        # calculate loss 
        loss = criterion(yhat, y)

        # calculate gradients of parameters 
        loss.backward()

        # update parameters 
        optimizer.step()

        
        #print("Batch {}, Batch_loss = {}".format(b_no, loss.data))
        loss_t+= loss.data
        
    loss_list.append(loss_t)
    print("Epoch {}, Epoch_loss = {}".format(epoch, loss_t))
    
    correct=0
    N_tst=0
    for x_test, y_test in validation_loader:
        # set model to eval 
        model.eval()
        
        #make a prediction 
        z=model(x_test)
        N_tst+=100
        #print(yhat, yhat.shape)

        #find max 
        yhat= torch.max(z.data, 1)[1]
        #print(yhat, yhat.shape)
       
        #Calculate misclassified  samples in mini-batch 
        #hint
        correct +=(yhat==y_test).sum().item()
        #print("Validation batch = {}, Accuracy = {}".format(N_tst/100, correct/N_tst))
   
    accuracy=correct/N_test
    accuracy_list.append(accuracy)
    print("Validation Epoch = {}, Accuracy = {}".format(epoch, accuracy))
    




Epoch 0, Epoch_loss = 1448.3280029296875
Validation Epoch = 0, Accuracy = 0.4952
Epoch 1, Epoch_loss = 1547.1043701171875
Validation Epoch = 1, Accuracy = 0.4924
Epoch 2, Epoch_loss = 1626.071044921875
Validation Epoch = 2, Accuracy = 0.4941
Epoch 3, Epoch_loss = 1624.0650634765625
Validation Epoch = 3, Accuracy = 0.4959
Epoch 4, Epoch_loss = 1625.549560546875
Validation Epoch = 4, Accuracy = 0.4967
Epoch 5, Epoch_loss = 1623.711181640625
Validation Epoch = 5, Accuracy = 0.4967
Epoch 6, Epoch_loss = 1623.407470703125
Validation Epoch = 6, Accuracy = 0.4974
Epoch 7, Epoch_loss = 1623.171875
Validation Epoch = 7, Accuracy = 0.4975
Epoch 8, Epoch_loss = 1623.1502685546875
Validation Epoch = 8, Accuracy = 0.4976
Epoch 9, Epoch_loss = 1622.8953857421875
Validation Epoch = 9, Accuracy = 0.4975
Epoch 10, Epoch_loss = 1622.69677734375
Validation Epoch = 10, Accuracy = 0.4975
Epoch 11, Epoch_loss = 1611.589111328125
Validation Epoch = 11, Accuracy = 0.5243
Epoch 12, Epoch_loss = 1481.3264160156